In [ ]:
# --- Imports ---
import pandas as pd
import numpy as np
import pickle
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW
from pyproj import Transformer
from scipy.linalg import LinAlgError
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

np.random.seed(42)

# --- Load Data ---
df_full = pd.read_csv('/content/MGWRwithPCA_042625_for_GWR.csv')

# --- Reproject ---
if df_full['X_Coor'].max() < 360 and df_full['Y_Coor'].max() < 90:
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
    xs, ys = transformer.transform(df_full['X_Coor'].values, df_full['Y_Coor'].values)
    df_full[['X_Coor', 'Y_Coor']] = np.vstack((xs, ys)).T

# --- Check for Mean_Elevation ---
if 'Mean_Elevation' not in df_full.columns:
    raise ValueError("Required column 'Mean_Elevation' not found in dataset.")

# --- Config ---
BOOTSTRAPS = 50
SAMPLE_SIZE = 10000
epsilon = 1e-6

# --- Regularize Function ---
def regularize_matrix(X, epsilon=1e-6):
    return X + np.random.normal(0, epsilon, X.shape)

# --- Storage ---
summary_records = []

# --- Bootstrap Loop ---
for run in range(1, BOOTSTRAPS + 1):
    print(f"\n=== Running Bootstrap {run}/{BOOTSTRAPS} ===")
    df = df_full.sample(SAMPLE_SIZE, random_state=run).dropna(subset=[
    'Temperature', 'Pct_Canopy_1', 'Built_Environment_Index', 'Mean_Elevation',
    'X_Coor', 'Y_Coor'
])

    coords = df[['X_Coor', 'Y_Coor']].values
    y = df[['Temperature']].values
    X = df[['Pct_Canopy_1', 'Built_Environment_Index', 'Mean_Elevation']].values

    try:
        selector = Sel_BW(coords, y, X, kernel='gaussian', fixed=False)
        bw = selector.search()
    except Exception as e:
        print(f"Bandwidth selection failed, regularizing predictors: {e}")
        X = regularize_matrix(X)
        selector = Sel_BW(coords, y, X, kernel='gaussian', fixed=False)
        bw = selector.search()

    print(f"Selected Bandwidth: {bw:.2f}")

    # GWR Fit
    try:
        model = GWR(coords, y, X, bw=bw, kernel='gaussian', fixed=False)
        res = model.fit()
    except LinAlgError:
        print("Singular matrix encountered during fitting. Regularizing predictors.")
        X = regularize_matrix(X)
        model = GWR(coords, y, X, bw=bw, kernel='gaussian', fixed=False)
        res = model.fit()

    # Build Output
    df_result = pd.DataFrame({
        'X_Coor': df['X_Coor'],
        'Y_Coor': df['Y_Coor'],
        'Observed_Temperature': df['Temperature'],
        'Predicted_Temperature': (
            res.params[:, 0] +
            res.params[:, 1] * df['Pct_Canopy_1'] +
            res.params[:, 2] * df['Built_Environment_Index'] +
            res.params[:, 3] * df['Mean_Elevation']
        ),
        'Intercept': res.params[:, 0],
        'Canopy_Coeff': res.params[:, 1],
        'BuiltEnv_Coeff': res.params[:, 2],
        'Elevation_Coeff': res.params[:, 3],
        'Local_R2': np.nan_to_num(res.localR2.flatten(), nan=np.nan)
    })

    df_result['Residuals'] = df_result['Observed_Temperature'] - df_result['Predicted_Temperature']

    # Save Each Run
    df_result.to_csv(f'/content/GWR_full_run_{run}.csv', index=False)

    # Save basic summary
    mean_r2 = np.nanmean(df_result['Local_R2'])
    rmse = np.sqrt(mean_squared_error(df_result['Observed_Temperature'], df_result['Predicted_Temperature']))
    mae = mean_absolute_error(df_result['Observed_Temperature'], df_result['Predicted_Temperature'])
    r2 = r2_score(df_result['Observed_Temperature'], df_result['Predicted_Temperature'])

    summary_records.append({
        'Run': run,
        'Bandwidth': bw,
        'Mean_Local_R2': mean_r2,
        'RMSE': rmse,
        'MAE': mae,
        'R2_Global': r2
    })

# Save all summaries
summary_df = pd.DataFrame(summary_records)
summary_df.to_csv('/content/GWR_bootstrap_summary_full.csv', index=False)

print("\n All 50 runs completed and saved.")

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

# === Load your combined GWR CSV ===
df = pd.read_csv('/content/GWR_all_runs_combined.csv')

# === Group by coordinates and compute average values across all runs ===
agg = df.groupby(['X_Coor', 'Y_Coor']).agg({
    'Canopy_Coeff': 'mean',
    'BuiltEnv_Coeff': 'mean',
    'Elevation_Coeff': 'mean',
    'Local_R2': 'mean',
    'Predicted_Temperature': 'mean',
    'Residuals': 'mean'
}).reset_index()

# === Convert to GeoDataFrame ===
gdf = gpd.GeoDataFrame(
    agg,
    geometry=gpd.points_from_xy(agg['X_Coor'], agg['Y_Coor']),
    crs='EPSG:3857'  # Matches most web basemaps
)

# === Plotting function with basemap and styling ===
def plot_map_with_basemap(gdf, column, title, cmap='coolwarm', save=False):
    fig, ax = plt.subplots(figsize=(12, 10))
    gdf.plot(
        column=column,
        cmap=cmap,
        ax=ax,
        legend=True,
        legend_kwds={'label': column.replace('_', ' ').title(), 'shrink': 0.7},
        markersize=8,
        alpha=0.9
    )
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, zoom=13)
    ax.set_title(title, fontsize=18)
    ax.axis('off')
    if save:
        plt.savefig(f"/content/{column}_styled_map.png", dpi=400, bbox_inches='tight')
    plt.show()

In [ ]:
# Tree Canopy Coefficient
plot_map_with_basemap(
    gdf,
    'Canopy_Coeff',
    'Tree Canopy Cooling Coefficient',
    cmap='Spectral',
    save=True
)

# Built Environment Coefficient
plot_map_with_basemap(
    gdf,
    'BuiltEnv_Coeff',
    'Built Environment Heat Coefficient',
    cmap='RdYlBu_r',
    save=True
)

# Elevation Coefficient
plot_map_with_basemap(
    gdf,
    'Elevation_Coeff',
    'Elevation Impact on Temperature',
    cmap='PiYG',
    save=True
)

# Predicted Temperature
plot_map_with_basemap(
    gdf,
    'Predicted_Temperature',
    'Predicted Ambient Temperature (°F)',
    cmap='inferno',
    save=True
)

# Model Residuals
plot_map_with_basemap(
    gdf,
    'Residuals',
    'Mean Model Residuals (°F)',
    cmap='coolwarm',
    save=True
)

# Local R²
plot_map_with_basemap(
    gdf,
    'Local_R2',
    'Local R² Across 50 Runs',
    cmap='viridis',
    save=True
)

In [ ]:
from esda.moran import Moran
import libpysal
import numpy as np

# --- Convert GeoDataFrame to projected CRS for spatial weights (already in EPSG:3857)
# Reuse `gdf` from earlier that contains residuals etc.

# --- Build spatial weights (k-nearest neighbors or distance band)
w = libpysal.weights.KNN.from_dataframe(gdf, k=8)
w.transform = 'r'  # Row-standardize

# --- Run Moran's I on Residuals
moran_res = Moran(gdf['Residuals'], w)

print(" Moran's I for Residuals:")
print(f"  Moran's I: {moran_res.I:.4f}")
print(f"  p-value:   {moran_res.p_sim:.4f}")

In [ ]:
import matplotlib.pyplot as plt
from splot.esda import moran_scatterplot
from esda.moran import Moran

# w = libpysal.weights.KNN.from_dataframe(gdf, k=8)
# moran_res = Moran(gdf['Residuals'], w)

fig, ax = moran_scatterplot(moran_res, aspect_equal=True)
ax.set_title("Moran’s I Scatterplot: Model Residuals", fontsize=16)
plt.show()

In [ ]:
from esda.moran import Moran_Local
from splot.esda import lisa_cluster

# Calculate local Moran's I
lisa = Moran_Local(gdf['Residuals'], w)

# Add classification labels to your GeoDataFrame
gdf['lisa_cluster'] = lisa.q
gdf['lisa_signif'] = lisa.p_sim < 0.05

# Plot LISA cluster map
fig, ax = lisa_cluster(lisa, gdf, p=0.05)
ax.set_title("LISA Cluster Map of Residuals", fontsize=16)
plt.show()

In [ ]:
# Load the original full dataset
df_obs = pd.read_csv('/content/MGWRwithPCA_042625_for_GWR.csv')

# Convert to GeoDataFrame
gdf_obs = gpd.GeoDataFrame(
    df_obs,
    geometry=gpd.points_from_xy(df_obs['X_Coor'], df_obs['Y_Coor']),
    crs='EPSG:3857'
)

# Plot the observed temps
plot_map_with_basemap(
    gdf_obs,
    'Temperature',
    'Observed Ambient Air Temperature (°F)',
    cmap='inferno',
    save=True
)

In [ ]:
from splot.esda import moran_scatterplot
import matplotlib.pyplot as plt

# Plot Moran’s I scatterplot
fig, ax = moran_scatterplot(moran_obs, aspect_equal=True)
ax.set_title("Moran’s I Scatterplot: Observed Temperature", fontsize=16)
ax.set_xlabel("Observed Temperature (°F)")
ax.set_ylabel("Spatial Lag of Temperature")
plt.show()

In [ ]:
from esda.moran import Moran_Local
from splot.esda import lisa_cluster

# Calculate local Moran’s I
lisa_obs = Moran_Local(gdf_obs_clean['Temperature'], w_obs)

# Plot LISA cluster map
fig, ax = lisa_cluster(lisa_obs, gdf_obs_clean, p=0.05)
ax.set_title("LISA Cluster Map: Observed Temperature", fontsize=16)
plt.show()

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

# Load combined data
df = pd.read_csv('/content/GWR_all_runs_combined.csv')

# Residual SD (volatility map)
resid_sd = df.groupby(['X_Coor', 'Y_Coor'])['Residuals'].std().reset_index(name='Residuals_SD')

# Coefficient SD maps
coef_sd = df.groupby(['X_Coor', 'Y_Coor'])[['Canopy_Coeff', 'BuiltEnv_Coeff', 'Elevation_Coeff']].std().reset_index()
coef_sd.rename(columns={
    'Canopy_Coeff': 'Canopy_SD',
    'BuiltEnv_Coeff': 'BuiltEnv_SD',
    'Elevation_Coeff': 'Elevation_SD'
}, inplace=True)

# Residual sign frequency map
df['Residual_Sign'] = df['Residuals'] > 0  # True = underprediction
sign_freq = df.groupby(['X_Coor', 'Y_Coor'])['Residual_Sign'].mean().reset_index(name='Underprediction_Frequency')

# Local R² (mean across runs, already done)
local_r2 = df.groupby(['X_Coor', 'Y_Coor'])['Local_R2'].mean().reset_index()

# Merge everything
merged = resid_sd.merge(coef_sd, on=['X_Coor', 'Y_Coor'])\
                 .merge(sign_freq, on=['X_Coor', 'Y_Coor'])\
                 .merge(local_r2, on=['X_Coor', 'Y_Coor'])

# Make GeoDataFrame
gdf_diag = gpd.GeoDataFrame(
    merged,
    geometry=gpd.points_from_xy(merged['X_Coor'], merged['Y_Coor']),
    crs='EPSG:3857'
)


In [ ]:
def plot_map_with_basemap(gdf, column, title, cmap='coolwarm', vmin=None, vmax=None, save=False):
    fig, ax = plt.subplots(figsize=(12, 10))
    gdf.plot(
        column=column,
        cmap=cmap,
        ax=ax,
        legend=True,
        vmin=vmin,
        vmax=vmax,
        legend_kwds={'label': column.replace('_', ' ').title(), 'shrink': 0.7},
        markersize=8,
        alpha=0.9
    )
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, zoom=13)
    ax.set_title(title, fontsize=18)
    ax.axis('off')
    if save:
        plt.savefig(f"/content/{column}_diagnostic_map.png", dpi=400, bbox_inches='tight')
    plt.show()


In [ ]:
# Residual standard deviation (volatility)
plot_map_with_basemap(
    gdf_diag,
    'Residuals_SD',
    'Residual Standard Deviation (Across Runs)',
    cmap='inferno',
    save=True
)

# Coefficient standard deviations
plot_map_with_basemap(
    gdf_diag,
    'Canopy_SD',
    'Tree Canopy Coefficient Std. Dev.',
    cmap='YlGnBu',
    save=True
)

plot_map_with_basemap(
    gdf_diag,
    'BuiltEnv_SD',
    'Built Environment Coefficient Std. Dev.',
    cmap='OrRd',
    save=True
)

plot_map_with_basemap(
    gdf_diag,
    'Elevation_SD',
    'Elevation Coefficient Std. Dev.',
    cmap='PuBuGn',
    save=True
)

# Residual sign frequency (bias map)
plot_map_with_basemap(
    gdf_diag,
    'Underprediction_Frequency',
    'Model Underprediction Frequency',
    cmap='coolwarm',
    vmin=0, vmax=1,
    save=True
)

# Local R² (just in case)
plot_map_with_basemap(
    gdf_diag,
    'Local_R2',
    'Mean Local R² Across 50 GWR Runs',
    cmap='viridis',
    vmin=0.4, vmax=0.5,
    save=True
)